In [ ]:
pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
# IMPORT THE LIBRARIES
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import json
import pprint

import os
import re
import requests
import time

In [ ]:
# Spotify API KEYS
client_id = 'f9d64942f5f74f3282916225d2f77630'
client_secret = 'cc318334877a40cfae2d59e07ae86d9f'
client_credentials_manager = SpotifyClientCredentials('f9d64942f5f74f3282916225d2f77630', 'cc318334877a40cfae2d59e07ae86d9f')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
# os.getcwd()
guitar = pd.read_csv('Dataset_Song_Features_clean.csv')
guitar

,Band,Song,Player,Technique,Difficulty,Score
0,The Beatles,Blackbird,Paul McCartney,"['Guitar', 'Acoustic Guitar (nylon)']",Intermediate,3/8
1,Isaac Albéniz,Asturias,Ýz 1,['Acoustic Guitar (nylon)'],Intermediate,5/8
2,AC/DC,Back In Black,Angus Young (Lead),['Distortion Guitar'],Intermediate,3/8
3,Pachelbel,Canon in D Major,NaN,['Acoustic Guitar (nylon)'],Intermediate,4/8
4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,['Acoustic Guitar (steel)'],Beginner,2/8
...,...,...,...,...,...,...
4738,Your Mumies,play.,NaN,['Distortion Guitar'],Intermediate,3/8
4739,Slaver,The moon,NaN,['Distortion Guitar'],Intermediate,3/8
4740,Vundabar,Alien Blues,NaN,['Distortion Guitar'],Intermediate,4/8
4741,Paul McCartney,Uncle Albert/Admiral Halsey,NaN,['Electric Guitar (jazz)'],Beginner,1/8


In [ ]:
df = guitar.copy() #copy
df.iloc[0:50]

,Band,Song,Player,Technique,Difficulty,Score
0,The Beatles,Blackbird,Paul McCartney,"['Guitar', 'Acoustic Guitar (nylon)']",Intermediate,3/8
1,Isaac Albéniz,Asturias,Ýz 1,['Acoustic Guitar (nylon)'],Intermediate,5/8
2,AC/DC,Back In Black,Angus Young (Lead),['Distortion Guitar'],Intermediate,3/8
3,Pachelbel,Canon in D Major,NaN,['Acoustic Guitar (nylon)'],Intermediate,4/8
4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,['Acoustic Guitar (steel)'],Beginner,2/8
5,Jimi Hendrix,Little Wing,Jimi Hendrix,['Electric Guitar (jazz) 1'],Intermediate,5/8
6,Pink Floyd,Shine On You Crazy Diamond,David Gilmour,['Overdriven Guitar'],Beginner,2/8
7,Kansas,Dust In The Wind,NaN,"['Acoustic Guitar (nylon)', 'Guitar 1']",Intermediate,3/8
8,Francisco Tarrega,Capricho Arabe,NaN,"['Acoustic Guitar (nylon)', 'Serenata para Gui...",Intermediate,5/8
9,Pachelbel,Canon In C,NaN,['Acoustic Guitar (nylon)'],Beginner,1/8


In [ ]:
# Define features columns

df["ID_TRACK"] = ""
df["POPULARITY"] = ""
df["RELEASE_DATE"] = ""
df["EXTERNAL_URLS"] = ""
df["SEARCH_URL"] = ""
df["AVAILABLE_MARKETS"] = ""       # NON CI INTERESSA ?    
df["danceability"] = ""
df["energy"] = ""
df["loudness"] = ""
df["speechiness"] = ""
df["acousticness"] = ""
df["instrumentalness"] = ""
df["liveness"] = ""
df["valence"] = ""
df["tempo"] = ""
df["duration_ms"] = ""
#df["key"] = ""
#df["mode"] = ""
#df["beats"] = ""

In [ ]:
def find_track_ID(artist, track,i):
        try:
                ## Remove apostrophes
                artist = re.sub(f'\'', '', artist)
                # print(artist)
                track = re.sub(f'\'', '', track)
                # print(track)
                # track_id = sp.search(q='artist:' + artist  + ' track:' + track, type='track')    
                track_id = sp.search(q=+ artist + ' - ' + track, type='artist, track')    
                # pprint.pprint(track_id)
                ID_TRACK = track_id['tracks']['items'][0]['id']
                #print(ID_TRACK)
                POPULARITY = track_id['tracks']['items'][0]['popularity']
                #print(POPULARITY)
                RELEASE_DATE = track_id['tracks']['items'][0]['album']['release_date']
                EXTERNAL_URLS = track_id['tracks']['items'][0]['album']['external_urls']['spotify']
                SEARCH_URL = track_id['tracks']['items'][0]['preview_url']
                # AVAILABLE_MARKETS = track_id['tracks']['items'][0]['album']['available_markets']

                pd.options.mode.chained_assignment = None
                df["ID_TRACK"][i] = ID_TRACK
                df["POPULARITY"][i] = POPULARITY
                df["RELEASE_DATE"][i] = RELEASE_DATE
                df["EXTERNAL_URLS"][i] = EXTERNAL_URLS
                df["SEARCH_URL"][i] = SEARCH_URL
                # df["AVAILABLE_MARKETS"][i] = AVAILABLE_MARKETS     

                #audio_features
                af = sp.audio_features(str(ID_TRACK))[0]
                
                df["danceability"][i]= af["danceability"]
                df["energy"][i] = af["energy"]
                df["loudness"][i] = af["loudness"]
                df["speechiness"][i] = af["speechiness"]
                df["acousticness"][i] = af["acousticness"]
                df["instrumentalness"][i] = af["instrumentalness"]
                df["liveness"][i] = af["liveness"]
                df["valence"][i] = af["valence"]
                df["tempo"][i] = af["tempo"]
                df["duration_ms"][i] = af["duration_ms"]

        except:
                # print(track, artist)
                track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track')    
                # pprint.pprint(track_id)
                ID_TRACK = track_id['tracks']['items'][0]['id']
                #print(ID_TRACK)
                POPULARITY = track_id['tracks']['items'][0]['popularity']
                #print(POPULARITY)
                RELEASE_DATE = track_id['tracks']['items'][0]['album']['release_date']
                EXTERNAL_URLS = track_id['tracks']['items'][0]['album']['external_urls']['spotify']
                SEARCH_URL = track_id['tracks']['items'][0]['preview_url']
                # AVAILABLE_MARKETS = track_id['tracks']['items'][0]['album']['available_markets']

                pd.options.mode.chained_assignment = None
                df["ID_TRACK"][i] = ID_TRACK
                df["POPULARITY"][i] = POPULARITY
                df["RELEASE_DATE"][i] = RELEASE_DATE
                df["EXTERNAL_URLS"][i] = EXTERNAL_URLS
                df["SEARCH_URL"][i] = SEARCH_URL
                # df["AVAILABLE_MARKETS"][i] = AVAILABLE_MARKETS     

                #audio_features
                af = sp.audio_features(str(ID_TRACK))[0]
                
                df["danceability"][i]= af["danceability"]
                df["energy"][i] = af["energy"]
                df["loudness"][i] = af["loudness"]
                df["speechiness"][i] = af["speechiness"]
                df["acousticness"][i] = af["acousticness"]
                df["instrumentalness"][i] = af["instrumentalness"]
                df["liveness"][i] = af["liveness"]
                df["valence"][i] = af["valence"]
                df["tempo"][i] = af["tempo"]
                df["duration_ms"][i] = af["duration_ms"]

                #audio_analysis                                   # NON FUNZIONA
#        aa = sp.audio_analysis(str(ID_TRACK))[0]

#        df["key"][i] = aa["key"]
#        df["mode"][i] = aa["mode"]
#        df["beats"][i] = aa["beats"]



In [ ]:
error = []
found = []
for i in range(len(df)): 
    try:
        find_track_ID(df["Band"].iloc[i],df["Song"].iloc[i], i)
        # print(i)
        found.append(i)
    except Exception as e: # work on python 3.x
        print(df["Band"].iloc[i], df["Song"].iloc[i])
        ## Print error
        print(e, i)
        # print("Error: element not found", i)
        ## Remove warning advise
        pd.options.mode.chained_assignment = None
        df["ID_TRACK"][i] = None
        df["POPULARITY"][i] = None
        df["RELEASE_DATE"][i] = None
        df["EXTERNAL_URLS"][i] = None
        df["SEARCH_URL"][i] = None
        # df["AVAILABLE_MARKETS"][i] = None
        df["danceability"][i]= None
        df["energy"][i] = None
        df["loudness"][i] = None
        df["speechiness"][i] = None
        df["acousticness"][i] = None
        df["instrumentalness"][i] = None
        df["liveness"][i] = None
        df["valence"][i] = None
        df["tempo"][i] = None
        df["duration_ms"][i] = None
        error.append(i)
    
    
print("")
print(f'{len(found)} elements found, {len(error)} elements not found')

Kaare Norge Nocturne Op. 9 No. 2 
list index out of range 15
Slash The Godfather Theme
list index out of range 23
Yiruma-Shunga Jung Fingerstyle River flows in you-Yiruma 
list index out of range 25
Theme Songs Pirates Of The Caribbean
list index out of range 29
JerryC Canon Rock
list index out of range 38
Alan Walker Faded Fingerstyle
list index out of range 58
Joe Hisaishi Spirited Away Ost - One Summer's Day
list index out of range 65
Eddie Van Der Meer Naruto Ost - Sadness & Sorrow 
list index out of range 69
Lemon Kenshi Yonezu 米津玄師
list index out of range 72
Eddie Van Der Meer Unravel - Tokyo Ghoul OP 1
list index out of range 75
Firelake Dirge For The Planet
list index out of range 80
Jaskier Toss A Coin To Your Witcher
list index out of range 82
Carlos Santana Smooth
list index out of range 83
The White Stripes Seven Nation Army Edit
list index out of range 84
Kotaro Oshio Twilight
list index out of range 93
Neil Wicker Shallow 
list index out of range 94
Kana-Boon Silhouette
l

The Damm Festival Turned To Slag
list index out of range 612
Therapy Hey Satan - You Rock
list index out of range 613
Therapy Rock You Monkeys
list index out of range 615
Therapy Perish The Thought
list index out of range 617
Nobuo Uematsu Bombing Mission  Prelude
list index out of range 618
Zombie Guts Brutal Death
list index out of range 619
Atherakia Burning Down The Opera  Dead And Buried - Episode II
list index out of range 621
Azyrius Fight To Survive
list index out of range 622
Brou On The Way To Disaster
list index out of range 623
Composers of MSB Pink Marshmallows
list index out of range 624
Heavy Competition 9 Charles's Redemption
list index out of range 625
Judgement Day The Crimson Moon
list index out of range 626
Kyuss Super Scooper
list index out of range 627
Lost Vagabond Vanity
list index out of range 628
MSB Competition Jan-Feb 2005 Ruins' Tale
list index out of range 629
Rr5rhoads Song 1
list index out of range 630
War Sucks Cruzifix
list index out of range 631
Jimi 

Face Of Death Meth Ampheticide
list index out of range 889
Dd dd unicorn
list index out of range 890
The Age Bench-press
list index out of range 894
Slayer Angel Of Death D Standard
list index out of range 895
Vektor Recharging The Void D Standard
list index out of range 897
Shawn Cochran Jurassic Park Theme 
list index out of range 898
Metallica Through The Never Drop D
list index out of range 901
Metallica Leper Messiah Drop D
list index out of range 902
Evgeni Vutov Pirate1
list index out of range 903
Sinister Desire Melancholy 
list index out of range 904
Face Of Death Misc Hellderado
list index out of range 905
Hellbroken A New Life Above... Or Below?
list index out of range 906
Hellbroken Dementia
list index out of range 907
Hellbroken Discontinued Thought Side 2
list index out of range 908
Hellbroken Discontinued Thought Side 3
list index out of range 909
Hellbroken Discontinued Thought Side 1
list index out of range 910
Hellbroken Atrocity Attraction
list index out of range 911

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Poison The Well track:Loved Ones Excerpts From Speeches Of How Great You Were And Will Never Be Agai', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


Poison The Well Loved Ones Excerpts From Speeches Of How Great You We're And Will Never Be Agai
http status: 404, code:-1 - https://api.spotify.com/v1/search?q=artist%3APoison+The+Well+track%3ALoved+Ones+Excerpts+From+Speeches+Of+How+Great+You+Were+And+Will+Never+Be+Agai&limit=10&offset=0&type=track:
 Not found., reason: None 1030
Wounds Panzer Attack
list index out of range 1034
Crucifire Moon Jaruth
list index out of range 1036
Ivariis L0d
list index out of range 1039
R8932jrj823 r328j989r32
list index out of range 1041
Mastodondon Roots Remain
list index out of range 1042
Gutspawn Storm of Iron
list index out of range 1044
Gutspawn Hell Pit Abomination
list index out of range 1045
Gutspawn Untitled
list index out of range 1046
Gutspawn In The Temple of the Horned Rat
list index out of range 1047
Southking Hellhound
list index out of range 1048
Speedhammer un
list index out of range 1049
A Band song that all instruments have a solo
list index out of range 1052
Metallica Orion Standar

...

Untitled WT5314
list index out of range 4088
Slipknot SICCCCC
list index out of range 4089
Synqqq Feat fit3
list index out of range 4091
Blood Laced Misery Untitled 30
list index out of range 4094
Shredagge Autumn Of My Steps
list index out of range 4095
Arc Systems Works Love the Subhuman Self
list index out of range 4099
Xy Jay The Garden Of Eden
list index out of range 4100
Deftones Rocket Skates Drop B
list index out of range 4101
Braille The Dreadman
list index out of range 4102
Electriccover WITD
list index out of range 4103
Pathaway I Am Not Your Daughter, I Am Not Your Son
list index out of range 4105
Untitled WT_827 Gui & Drm
list index out of range 4106
Pathaway Only The Beginning
list index out of range 4107
Untitled WT1512
list index out of range 4108
Xy Jay What Lies Beyond Strength
list index out of range 4109
Mick Thomson 2000 Soundcheck Riff
list index out of range 4112
Untitled Wt1839
list index out of range 4113
Untitled StrangeMusik4
list index out of range 4114
Pocu

Zigzag subaru
list index out of range 4330
Bkmz hueta2
list index out of range 4332
Konomi Suzuki Absolute Duo
list index out of range 4333
Deftones Bumble D
list index out of range 4335
Idioty The Last Limit
list index out of range 4336
Reality Spill Guillermo
list index out of range 4341
Warhound Poor Little Me
list index out of range 4342
Alex Post grunge appreciation
list index out of range 4343
Warhound LoveSong
list index out of range 4344
Idioty Time Of The World
list index out of range 4348
Hate The Thought Misplaced 
list index out of range 4349
Hate The Thought Floral 
list index out of range 4350
Dark Matter fireflies practice
list index out of range 4351
Creatures Of Furkahn Lavender
list index out of range 4353
Weywot pisa, the leaning tower of
list index out of range 4354
Warhound Break My Heart
list index out of range 4357
A Cool Guy lk
list index out of range 4361
Winterstone Otherside 
list index out of range 4363
Meme School PUSSY!
list index out of range 4364
Warhoun

Tom Donahue Bullet With Butterfly Wings
list index out of range 4539
Tom Donahue Knockin' On Heaven's Door
list index out of range 4541
Vlad I Vadim Freezing
list index out of range 4542
Zxco zxco
list index out of range 4543
Sorry Stereo K*** Myself Drunk
list index out of range 4544
Qwr qwr
list index out of range 4550
Lampadina knup ozzep
list index out of range 4553
Tyoma The Lion's Den
list index out of range 4554
Halloween Night Misc Bridge I
list index out of range 4555
Arc Systems Works The Disaster of Passion
list index out of range 4557
Jonathan Heavy Metalcore 1
list index out of range 4558
Metallica Havester Of Sorrow no full
list index out of range 4560
Depressed Sandman k no drums
list index out of range 4561
Metallica Damage.Inc 
list index out of range 4562
Misfits American Psycho 2
list index out of range 4563
Lucas Avila (Upload) Unravel
list index out of range 4564
Lucas Avila (Upload) Enigmatic Feeling
list index out of range 4565
Lucas Avila (Upload) Telecastic fak

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Continuous Atrocities Against The Condemned Species For The Purpose Of Self-Preservation From The Inevitable Consequences Of Playing God Infront The Sights Of Nature And Existance track:Rage...Pure Unfiltered, Uncontrollable And Unstoppable Rage Spewing From The Blood Vessels, Causing Shaking Within The Body', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Continuous Atrocities Against The Condemned Species For The Purpose Of Self-Preservation From The Inevitable Consequences Of Playing God Infront The Sights Of Nature And Existance track:Witnessing Every Human Act Of Disrespect Towards Their Settlement As The Old Man Barely Breathes With His Lungs, His Cries Remain Unheard Because His Neighbours Never Went Through The Path He Chose', 'limit': 10, 'offset': 0, 'type': 'track', 'ma

All Animals as Leaders - The Price of Everything and the Value of Nothing
list index out of range 4680
Genshin magatsumi takenaru kaminomikoto
list index out of range 4681
Continuous Atrocities Against The Condemned Species For The Purpose Of Self-Preservation From The Inevitable Consequences Of Playing God Infront The Sights Of Nature And Existance Rage...Pure Unfiltered, Uncontrollable And Unstoppable Rage Spewing From The Blood Vessels, Causing Shaking Within The Body
http status: 404, code:-1 - https://api.spotify.com/v1/search?q=artist%3AContinuous+Atrocities+Against+The+Condemned+Species+For+The+Purpose+Of+Self-Preservation+From+The+Inevitable+Consequences+Of+Playing+God+Infront+The+Sights+Of+Nature+And+Existance+track%3ARage...Pure+Unfiltered%2C+Uncontrollable+And+Unstoppable+Rage+Spewing+From+The+Blood+Vessels%2C+Causing+Shaking+Within+The+Body&limit=10&offset=0&type=track:
 Not found., reason: None 4682
Continuous Atrocities Against The Condemned Species For The Purpose Of Sel

In [ ]:
df = df.drop(['Player', 'Technique', 'Difficulty', 'Score'], axis = 1)
print('Found:', len(df[df['ID_TRACK'].notnull()]), len(guitar), len(df[df['ID_TRACK'].isnull()])/len(guitar)*100)
print('Found with preview url:', len(df[df['SEARCH_URL'].notnull()]), len(guitar), (len(df[df['SEARCH_URL'].isnull()])/len(guitar))*100)
df.head(2)
a=df[df['SEARCH_URL'].notnull()]

Found: 2283 4743 51.865907653383935
Found with preview url: 1899 4743 59.96204933586338


,Band,Song,ID_TRACK,POPULARITY,RELEASE_DATE,EXTERNAL_URLS,SEARCH_URL,AVAILABLE_MARKETS,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,The Beatles,Blackbird,5jgFfDIR6FR0gvlA56Nakr,72,1968-11-22,https://open.spotify.com/album/1klALx0u4AavZNE...,None,,0.686,0.127,-14.361,0.0342,0.754,0.000014,0.0573,0.372,93.699,138387
1,Isaac Albéniz,Asturias,2u6GTNZF07O4mmFg8CZozY,36,1981,https://open.spotify.com/album/5lY62FNmxjms4v3...,https://p.scdn.co/mp3-preview/5e9af1c3d16503e3...,,0.32,0.171,-20.784,0.0594,0.967,0.884,0.104,0.217,112.092,372733


In [ ]:
a['Band'] = a['Band'].apply(lambda x: re.sub(' $', '', x))
a['Song'] = a['Song'].apply(lambda x: re.sub(' $', '', x))

a.drop_duplicates(subset=['Band', 'Song', 'SEARCH_URL'], keep='first', inplace = True)
a.reset_index(inplace=True)
a.drop('index', inplace=True, axis = 1)
a.to_csv('spotify_features.csv', index = False)


a.to_csv('spotify_features.csv') 

In [41]:
df = pd.read_csv('/content/drive/MyDrive/DataMan_Songs/Dataset/spotify_features.csv')
df.sort_values(by=['Band', 'Song'])

,Band,Song,ID_TRACK,POPULARITY,RELEASE_DATE,EXTERNAL_URLS,SEARCH_URL,AVAILABLE_MARKETS,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1438,1,Otto,3NCgGRY9whGsiSHLvfLEnz,18,2021-07-30,https://open.spotify.com/album/2yORiycMziMpcyY...,https://p.scdn.co/mp3-preview/cd87d8db2eea58a8...,NaN,0.461,0.8930,-4.150,0.0646,0.003620,0.184000,0.3920,0.1850,100.077,537403
1712,100 Gecs,Stupid Horse,23Dapn8107GgxvXDIsPwWm,58,2019-05-31,https://open.spotify.com/album/2uhB1KivbFnlkAR...,https://p.scdn.co/mp3-preview/6cab4d3e8dfac7d8...,NaN,0.526,0.9020,-4.876,0.3140,0.052900,0.000000,0.5310,0.7970,168.188,121682
1310,10cm,c,6k6Llm7Kw3Ad1Xo1e6sIcR,40,2011-02-10,https://open.spotify.com/album/2NQxbsV4UYXegJQ...,https://p.scdn.co/mp3-preview/7f643d4fdeaaef3d...,NaN,0.759,0.3330,-8.680,0.0357,0.246000,0.000000,0.2220,0.6730,121.831,172760
1791,12,123,2nbvtZwRHAQ9nu4dVdpSCT,4,2022-01-14,https://open.spotify.com/album/3gDj9NyvNUd3xT0...,https://p.scdn.co/mp3-preview/c5769e3b526fdbab...,NaN,0.596,0.7260,-5.835,0.0448,0.058600,0.000664,0.2690,0.9060,164.168,248209
307,12,23,5Xo5TKZdvZwMbZsGbUjSYN,16,2020-12-23,https://open.spotify.com/album/7EMMKRBl7UZE59U...,https://p.scdn.co/mp3-preview/7a2a1babdd26d2c2...,NaN,0.796,0.4070,-8.248,0.2780,0.157000,0.003080,0.3800,0.2490,79.992,144065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,ZZ Top,La Grange,70YvYr2hGlS01bKRIho1HM,76,1973-07-26,https://open.spotify.com/album/0Em8m9kRctyH9S3...,https://p.scdn.co/mp3-preview/b7e2e1d3d033dae1...,NaN,0.545,0.6400,-9.937,0.2240,0.002490,0.030400,0.1960,0.6060,80.544,230480
143,ZZ Top,Sharp Dressed Man,1UBQ5GK8JaQjm5VbkBZY66,73,1983-03-23,https://open.spotify.com/album/5LMGAYhn2ywaxGZ...,https://p.scdn.co/mp3-preview/e3f0e4d07846c4d0...,NaN,0.601,0.8590,-5.263,0.0276,0.000359,0.001240,0.0871,0.4460,125.145,258027
1776,Zakk Wylde,Lost Prayer,6OUpPRqpE4WaA6F80J08SG,45,2016-05-06,https://open.spotify.com/album/2FVkTg0WnsEfXhn...,https://p.scdn.co/mp3-preview/bc8e3871441b4527...,NaN,0.596,0.5710,-7.821,0.0248,0.302000,0.000026,0.0934,0.3910,101.930,267870
1239,Zun,Bad Apple,6BOZISCnSSgm0N7sdPBckD,30,2006-05-21,https://open.spotify.com/album/0Vib5xCJHrjHLIX...,https://p.scdn.co/mp3-preview/ba9cc9c7d8d6bc87...,NaN,0.538,0.9700,-2.366,0.0805,0.002080,0.847000,0.0339,0.9640,160.801,195187


In [42]:
# WAV

# ## Define a function:
# - Input = artist name, song name, url preview
# - Output = Database(columns = 'Artist name', 'Song name', 'song.wav'

def downloader_wav(artist, song, url):
    
    wav_song = artist.replace(' ', '_') + '-' + song.replace(' ', '_') + '.wav'
    
    try:
        sample = requests.get(url)
        open(wav_song, 'wb').write(sample.content)
    except:
        pass
    
    
#     return(wav_song)

In [ ]:
start = time.perf_counter()
for track in range(len(df)):
    downloader_wav(df['Band'][track], df['Song'][track], df['SEARCH_URL'][track])
print(f'seconds elapsed: {time.perf_counter()-start}')

In [ ]:
## Move to Audio5
# len(os.listdir(os.getcwd()))
import shutil
for el in os.listdir(os.getcwd()):  
  if el.endswith(".wav"):
    colab_link = "/content/{}".format(el)
    gdrive_link = "/content/drive/MyDrive/DataMan_Songs/PROVA_SARA/"
    shutil.copy(colab_link, gdrive_link)

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/DataMan_Songs/PROVA_SARA')))

1798
